# Meteorology of Swindon

This script extracts and interprets meteorological data in Swindon. The ERA5 Land Monthly Averaged data set retrieved from Copernicus and covers a spatial regions of North 51.6°, West -1.9°, South 51.5°, East -1.7°.

In [30]:
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
from netCDF4 import Dataset, num2date
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cdsapi
import os

## 1. Data Extraction

Below is the code fow downloading the data from Copernicus. Once it is downloaded the rest of the script can run. To begin, where should we put the data?

In [20]:
data_file = "datafile"

In [21]:
def get_data(file_location):
    """ This method downloads the required data form Copernicus. """
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land-monthly-means',
        {
            'product_type': 'monthly_averaged_reanalysis',
            'variable': [
                '2m_temperature', 'snowfall', 'total_precipitation',
            ],
            'year': [
                '1981', '1982', '1983',
                '1984', '1985', '1986',
                '1987', '1988', '1989',
                '1990', '1991', '1992',
                '1993', '1994', '1995',
                '1996', '1997', '1998',
                '1999', '2000', '2001',
                '2002', '2003', '2004',
                '2005', '2006', '2007',
                '2008', '2009', '2010',
                '2011', '2012', '2013',
                '2014', '2015', '2016',
                '2017', '2018', '2019',
                '2020',
            ],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'time': '00:00',
            'area': [
                51.6, -1.9, 51.5,
                -1.7,
            ],
            'format': 'netcdf',
        },
        file_location)
get_data(data_file)

2021-01-09 11:13:25,899 INFO Welcome to the CDS
2021-01-09 11:13:25,901 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land-monthly-means
2021-01-09 11:13:26,057 INFO Request is completed
2021-01-09 11:13:26,059 INFO Downloading https://download-0009.copernicus-climate.eu/cache-compute-0009/cache/data9/adaptor.mars.internal-1610190167.965858-23345-17-ce80eb69-86b0-41cd-82f2-307ac6eb8382.nc to datafile (20.3K)
2021-01-09 11:13:26,218 INFO Download rate 129.1K/s                                                                    


Next, I extract the data into data structures which are easier to manage.

In [39]:
def refine_data(data_file):
    """ Extracts data from the data file and createss a dataframe from it. """
    ds = Dataset("{}.nc".format(data_file))
    # Get the dates (months) the data covers
    times = ds.variables['time']
    times = num2date(times[:], times.units)
    # Temperature, Rainfall and Snowfall needs averaging
    # in the spatial domain.
    temp = np.mean(ds.variables['t2m'][:], axis=(1, 2)) - 273.15 # Convert to celsius
    rain = np.mean(ds.variables['tp'][:], axis=(1, 2))
    snow = np.mean(ds.variables['sf'][:], axis=(1, 2))
    # Create pandas dataframe
    df = pd.DataFrame(data={'date': times, 'temp': temp, 'snow': snow, 'rain': rain})
    return df
df = refine_data(data_file)